In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.sparse import csr_matrix
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
def generate_maze(width, height, density):
    # Initialize the maze with walls
    maze = np.ones((height, width), dtype=np.int8)
    
    # Function to carve paths in the maze
    def carve(x, y):
        directions = [(2, 0), (-2, 0), (0, 2), (0, -2)]
        random.shuffle(directions)
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 1 <= nx < width - 1 and 1 <= ny < height - 1 and maze[ny][nx] == 1:
                maze[ny][nx] = 0
                maze[ny - dy//2][nx - dx//2] = 0
                carve(nx, ny)
    
    # Start carving from the top-left corner
    start_x, start_y = 1, 1
    maze[start_y][start_x] = 0
    carve(start_x, start_y)
    
    # Ensure the entrance and exit are open
    maze[1][0] = 0  # Entrance
    maze[height-2][width-1] = 0  # Exit
    
    # Add extra walls based on the density
    wall_count = int(density * (width * height) // 2)
    for _ in range(wall_count):
        wx, wy = random.randint(1, width - 2), random.randint(1, height - 2)
        if maze[wy][wx] == 0:
            maze[wy][wx] = 1
    
    return maze

In [ ]:
def visualize_maze(maze):
    plt.figure(figsize=(10, 10))
    plt.imshow(maze, cmap='binary')
    plt.xticks([]), plt.yticks([])  # Hide the axis
    # plt.show()
    plt.savefig("maze.pdf")

In [ ]:
def write_dense_matrix_as_petsc_binary(matrix, filename):
    """write dense matrix in petsc binary format

    Args:
        matrix (ndarray): 2d-array to be written
        filename (string): filename
    """
    # Convert the dense matrix to CSR format
    sparse_matrix = csr_matrix(matrix)
    sparse_matrix.sort_indices()

    with open(filename, "wb") as f:
        f.write(b"\x00\x12\x7b\x50")  # PETSc binary matrix class id
        f.write(np.array(sparse_matrix.shape, dtype=">i").tobytes())  # rows and cols
        f.write(np.array(sparse_matrix.nnz, dtype=">i").tobytes())  # nnz
        f.write(np.array(np.diff(sparse_matrix.indptr), dtype=">i").tobytes())  # row pointer
        f.write(np.array(sparse_matrix.indices, dtype=">i").tobytes())  # column indices
        f.write(np.array(sparse_matrix.data, dtype=">d").tobytes())  # values

    # with open(filename + ".info", "wb") as f:
    #     pass  # avoid PETSc complaints

In [ ]:
def maze2stagecosts(maze):
    # Exit has cost of -100
    # Walls have cost of np.inf
    # all other 0
    # copy maze to dtype double
    stage_costs = maze.astype(np.float64)
    stage_costs[stage_costs == 1] = 1e20
    stage_costs[stage_costs == 0] = 0.0
    stage_costs[height-2][width-1] = -100.0

    # state index = row-major index
    # flatten to vector in row-major order
    stage_costs = stage_costs.flatten()

    # stack vector horizontally to form a matrix (n x 5) [stay, north, east, south, west]
    stage_costs = np.stack([stage_costs, stage_costs, stage_costs, stage_costs, stage_costs], axis=1)

    return stage_costs

### Input here

In [ ]:
width = 25
height = 25
density = 0

# Ensure the width and height are odd
if width % 2 == 0:
    width += 1
if height % 2 == 0:
    height += 1

maze = generate_maze(width, height, density)
visualize_maze(maze)

In [ ]:
g = maze2stagecosts(maze)


In [ ]:
write_dense_matrix_as_petsc_binary(g, f"data/maze_{width}x{height}.bin")

### Postprocessing

In [ ]:
# Read the cost data and reshape to 2D matrix
costs = np.loadtxt("maze_cost.out").reshape(height, width)

# Read the policy data and reshape to 2D matrix
policy = np.loadtxt("maze_policy.out").reshape(height, width)

# Mask or threshold the wall costs
wall_cost_threshold = 1e10  # Define a threshold for wall costs
wall_mask = costs >= wall_cost_threshold
costs_masked = np.ma.masked_where(wall_mask, costs)

# Optionally, clip the costs to the desired range
min_cost, max_cost = -10000, -5000
costs_clipped = np.clip(costs_masked, min_cost, max_cost)

# Define the arrow directions for the policy
directions = {
    0: (0, 0),   # stay
    1: (-1, 0),  # north
    2: (0, 1),   # east
    3: (1, 0),   # south
    4: (0, -1)   # west
}

# Create a grid of coordinates
X, Y = np.meshgrid(np.arange(width), np.arange(height))

# Create arrays for the arrow directions
U = np.zeros_like(policy, dtype=float)
V = np.zeros_like(policy, dtype=float)

for action, (dy, dx) in directions.items():
    mask = (policy == action)
    U[mask] = dx
    V[mask] = dy

# Mask the arrows where there are walls
U = np.ma.masked_where(wall_mask, U)
V = np.ma.masked_where(wall_mask, V)

# Create a custom colormap from fully transparent to black
colors = [(0, 0, 0, 0), (0, 0, 0, 1)]  # RGBA for transparent to black
n_bins = 100  # Use 100 discrete levels
cmap_name = 'transparent_to_black'
cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

# Visualize the costs
plt.figure(figsize=(10, 10))

# Overlay black walls first with the custom colormap
plt.imshow(wall_mask, cmap=cm, interpolation='none', zorder=0)

# Then, overlay the costs
plt.imshow(costs_clipped, cmap='viridis', interpolation='none', zorder=1)
plt.colorbar(label='Costs')
plt.title('Maze Costs and Policy Visualization')
plt.xticks([]), plt.yticks([])  # Hide the axis

# Finally, overlay the policy arrows with smaller size, hiding arrows on walls
plt.quiver(X, Y, U, V, color='red', angles='xy', scale_units='xy', scale=4, width=0.007, zorder=2)

plt.savefig("maze_policy.pdf")